In [ ]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint

import nltk
nltk.download('punkt')

from gensim import models

In [ ]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'ENTAILMENT': 0, 'CONTRADICTION': 1, 'NEUTRAL': 2}

In [ ]:
# Function that reads data and parses data from file

fileName = 'SICK.txt'

with open(fileName, 'r') as f:
    Rows = [row.split('\t') for row in f.readlines()[1:]]

trainRows = [row for row in Rows if row[11]=='TRAIN\n']
testRows = [row for row in Rows if row[11]=='TEST\n']
validationRows = [row for row in Rows if row[11]=='TRIAL\n']

trainPremises = [extract(row[1]) for row in trainRows if row[3] in labels]
trainHypotheses = [extract(row[2]) for row in trainRows if row[3] in labels]
trainLabels = [labels[row[3]] for row in trainRows if row[3] in labels]

trainData = [trainPremises, trainHypotheses, trainLabels]

testPremises = [extract(row[1]) for row in testRows if row[3] in labels]
testHypotheses = [extract(row[2]) for row in testRows if row[3] in labels]
testLabels = [labels[row[3]] for row in testRows if row[3] in labels]

testData = [testPremises, testHypotheses, testLabels]

validationPremises = [extract(row[1]) for row in validationRows if row[3] in labels]
validationHypotheses = [extract(row[2]) for row in validationRows if row[3] in labels]
validationLabels = [labels[row[3]] for row in validationRows if row[3] in labels]

validationData = [validationPremises, validationHypotheses, validationLabels]

f.close()

In [ ]:
# Hyperparameters

maxLen = 32
epochs = 1000
batchSize = 512
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [ ]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [ ]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [ ]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [ ]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [ ]:
embeddingsDict = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# embeddingsDict = dict()
# glove = open(r'/content/drive/MyDrive/glove.840B.300d.txt', encoding='utf8')

# for line in glove:
#     records = line.split()
#     word = ''.join(records[:-300])
#     vectorDimensions = asarray(records[-300:], dtype='float32')
#     embeddingsDict[word] = vectorDimensions

# glove.close()

In [ ]:
# Iterate through the embeddings and store only those that are present in our vocabulary
'''
embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec
'''

embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    if index % 2500 == 0:
        print(index)
    if word in embeddingsDict.index_to_key:
        embeddingsMat[index] = embeddingsDict[word]

In [ ]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

In [ ]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

#rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
#rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

rnnH = LSTM(hiddenDimension, dropout=0.2, return_state=True)
rnnP = LSTM(hiddenDimension, dropout=0.2)

In [ ]:
# Apply batch normalization to the two input embeddings separately

premInput, sH, sC = rnnH(premInput)
hypoInput = rnnP(hypoInput, initial_state=[sH, sC])
premInput = tf.keras.layers.BatchNormalization()(premInput)
result = tf.keras.layers.BatchNormalization()(hypoInput)

In [ ]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
#joint = keras.layers.concatenate([premInput, hypoInput])
result = Dropout(0.2)(result)
'''
for i in range(3):
    result = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
    result = Dropout(0.2)(result)
    result = tf.keras.layers.BatchNormalization()(result)
'''

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function

result = Dense(hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
pred = Dense(3, activation='softmax')(result)

In [ ]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
mcCallback = ModelCheckpoint('./2lstm', monitor='val_loss')
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback, mcCallback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
9/9 [==============================] - ETA: 0s - loss: 0.2459 - accuracy: 0.8970

INFO:tensorflow:Assets written to: ./2lstm/assets


INFO:tensorflow:Assets written to: ./2lstm/assets


9/9 [==============================] - 19s 2s/step - loss: 0.2459 - accuracy: 0.8970 - val_loss: 2.2906 - val_accuracy: 0.4606
Epoch 2/1000
9/9 [==============================] - ETA: 0s - loss: 0.2657 - accuracy: 0.8869

INFO:tensorflow:Assets written to: ./2lstm/assets


...

INFO:tensorflow:Assets written to: ./2lstm/assets


9/9 [==============================] - 20s 2s/step - loss: 0.1578 - accuracy: 0.9329 - val_loss: 2.0993 - val_accuracy: 0.5455
Epoch 59/1000
9/9 [==============================] - ETA: 0s - loss: 0.1485 - accuracy: 0.9423

INFO:tensorflow:Assets written to: ./2lstm/assets


INFO:tensorflow:Assets written to: ./2lstm/assets


9/9 [==============================] - 19s 2s/step - loss: 0.1485 - accuracy: 0.9423 - val_loss: 2.1050 - val_accuracy: 0.5495


In [ ]:
model.save('./32TwoLSTM')

In [30]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

20/20 [==============================] - 5s 230ms/step - loss: 1.9728 - accuracy: 0.5514
Loss =  1.9727534055709839
Acc =  0.5513656735420227


In [ ]:
ccc = keras.models.load_model('./32TwoLSTM')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)